In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
# read dataset into datasets
from datasets import Dataset
import pandas as pd
dataset_name = "five-star-trajectories"
dataset_path = Path(f"lm_act_eval/.cache/{dataset_name }")
traj_df = pd.read_csv(dataset_path/"csv/data.csv")
traj_dataset = Dataset.from_pandas(traj_df)
print(traj_df.columns)
traj_df[traj_df.session_id==traj_df.session_id.unique()[0]]
traj_df.head(2)

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['session_id', 'idx_in_session', '_id', 'event_id', 'parent_id',
       'event_type', 'event_name', 'config', 'children_ids', 'inputs',
       'outputs', 'error', 'start_time', 'end_time', 'metadata', 'feedback',
       'metrics', 'user_properties', 'source', 'duration', 'project_id',
       'tenant', 'QUERY', 'URL', 'DOM', 'RULES', 'USER_CONTEXT',
       'PREV_ACTIONS', 'chat_history', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'ground_truth', 'screenshot', 'prompt', 'internal_metadata'],
      dtype='object')


,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,PREV_ACTIONS,chat_history,CURRENT_TIME,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata
0,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,0,65f0c5d705cef8bdb70153a9,cc3d0d22-4759-43d8-a688-d174ca94f9dd,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'Find a restaurant with vegetarian o...,...,NaN,[],"Tuesday, March 12, 2024 at 4:14:11 PM CDT","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN: To find a restaurant with vegetarian opt...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN
1,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,1,65f0c60705cef8bdb70153ab,44b78fc2-24e2-40e5-b39d-e887a7129587,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'Find a restaurant with vegetarian o...,...,- I am searching for restaurants with vegetari...,"[{'role': 'user', 'content': 'Find a restauran...","Tuesday, March 12, 2024 at 4:15:02 PM CDT","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...","The search for ""vegetarian"" in St. Louis has y...",https://multion-client-screenshots.s3.us-east-...,NaN,NaN


In [6]:
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  
  
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible

In [ ]:
# for i in [
#   traj_df.ground_truth,
#   traj_df.chat_completion_messages,
#   traj_df.screenshot
#   ]:
#   print(i[0].replace('\n\n','\n'))
#   print("")
  

In [ ]:
from typing import Callable
from tqdm import tqdm

In [ ]:
def criteria(eligibility_func: callable, cols: list, description: str=None):
  return pd.Series([eligibility_func(item) for item in tqdm(traj_df[cols], desc=description)])
def is_eligible():  
  # Apply the function to each URL in the column
  is_screenshot_accessible = criteria(is_screenshot_url_accessible, 'screenshot',"determine URL accessibility")
  return is_screenshot_accessible

eligible = is_eligible()
# Calculate the percentage of accessible URLs
percentage_accessible = (eligible.sum() / len(traj_df)) * 100
print(percentage_accessible)


determine URL accessibility: 100%|██████████| 3382/3382 [1:02:27<00:00,  1.11s/it]

4.376108811354229


In [ ]:
from finetuning.finetuning.evaluate.evaluators.vision import GPTV 
from finetuning.finetuning.evaluate.evaluators.vision.config import gptv_config
gptv_pipeline = GPTV(gptv_config)

<class 'finetuning.finetuning.evaluate.evaluators.vision.config.gptv_config'>


In [ ]:
from transformers import PreTrainedModel, PretrainedConfig
from transformers.modeling_outputs import BaseModelOutput
import requests

class CustomAPIModel(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self, config, api_endpoint):
        super().__init__(config)
        # Initialize any necessary components of the Hugging Face model here.
        self.api_endpoint = api_endpoint

    def forward(self, payload ):
        """
        # Here, instead of the traditional forward pass, we're making an API call.
        # Construct the payload for the API call.
        """
        # Make the API call
        response = requests.post(
          self.api_endpoint, json=payload)

        if response.status_code == 200:
            api_result = response.json()
            # You can process the result from the API as needed here.
            # For simplicity, we're returning it directly.
            return BaseModelOutput(
                last_hidden_state=api_result  # Placeholder: adapt as necessary based on the API's response structure
            )
        else:
            raise Exception(f"API call failed with status code {response.status_code}")

    @classmethod
    def from_pretrained(cls, *args, **kwargs):
        # Override the from_pretrained method if necessary.
        # This is where you'd handle loading pre-trained weights, which may not be applicable in this custom setup.
        # For simplicity, we'll just return an instance without loading pre-trained weights.
        config = PretrainedConfig()
        return cls(config)

In [ ]:
eligible_traj_df = traj_df[eligible]
len(eligible_traj_df)

In [ ]:
groundtruth, chat_completions, screenshots = eligible_traj_df.ground_truth, eligible_traj_df.chat_completion_messages, eligible_traj_df.screenshot
eligible_idx = eligible_traj_df.index

In [ ]:
chat_completions_processed = chat_completions.apply(lambda s: ParseChatCompletion().parse_as_json(s))

In [ ]:
traj_df.chat_completion_messages.apply(lambda s: ParseChatCompletion().parse_as_json(s))[0]

'**Goal**: Let\'s play a game - You are an expert agent named MULTI·ON developed by "MultiOn" controlling a browser (you are not just a language model anymore). \n\nYou are given:\n1. An objective that you are trying to achieve\n2. The URL of your current web page\n3. A simplified text description of what\'s visible in the browser window (more on that below)\n\n**Actions**:\nChoose from these actions: COMMANDS, ANSWER, or ASK_USER_HELP. If the user seeks information and you know the answer based on prior knowledge or the page content, answer without issuing commands.\n\n1. COMMANDS: Start with “COMMANDS:\n”. Use simple commands like CLICK <id>, TYPE <id> "<text>", or SUBMIT <id>. <id> is a number for an item on the webpage. After commands, write an explanation with "EXPLANATION: I am" followed by a summary of your goal (do not mention low-level details like IDs). Each command should be on a new line. In outputs, use only the integer part of the ID, without brackets or other characters 

In [ ]:
import warnings

completion_responses = []
for i in tqdm(eligible_idx, desc="Generating completions"):
  completion = gptv_pipeline.generate_completion(
    chat_completions_processed[i],
    screenshots[i]
  )
  if len(completion) > 1:
    warnings.warn("Completion length more than 0 is unexpected")
  completion_responses.append(completion[0].message.content)

In [ ]:
eligible_traj_df.loc[:, 'GPTV response'] = completion_responses

In [ ]:
traj_df_new = traj_df.merge(eligible_traj_df, how='left')
eligible_traj_df.to_csv(dataset_path/'csv/data+gptv-eligible.csv')
traj_df_new.to_csv(dataset_path/'csv/data+gptv.csv')

Logging

In [ ]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [ ]:
eligible_traj_df = pd.read_csv(dataset_path/'csv/data+gptv-eligible.csv')

In [ ]:
opentable_artifact = wandb.Artifact("opentable_trajectories_gptv.v0", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(opentable_table_eligible, "opentable-eligibleonly")
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/'csv/data+gptv-eligible.csv'))


wandb: WARNING Serializing object of type str that is 531752 bytes
wandb: WARNING Serializing object of type str that is 517908 bytes
wandb: WARNING Serializing object of type str that is 541284 bytes
wandb: WARNING Serializing object of type str that is 525810 bytes
wandb: WARNING Serializing object of type str that is 116864 bytes
wandb: WARNING Serializing object of type str that is 121274 bytes
wandb: WARNING Serializing object of type str that is 120688 bytes
wandb: WARNING Serializing object of type str that is 124364 bytes
wandb: WARNING Serializing object of type str that is 127354 bytes
wandb: WARNING Serializing object of type str that is 129308 bytes
wandb: WARNING Serializing object of type str that is 129582 bytes
wandb: WARNING Serializing object of type str that is 135310 bytes
wandb: WARNING Serializing object of type str that is 137856 bytes
wandb: WARNING Serializing object of type str that is 137644 bytes
wandb: WARNING Serializing object of type str that is 136800 b

ArtifactManifestEntry(path='data+gptv-eligible.csv', digest='ntwXQ/XrJitf5lp3JjCkKA==', size=14147166, local_path='/home/opadmin/.local/share/wandb/artifacts/staging/tmpqsma_gqk')

In [ ]:
# Start a W&B run to log data
run = wandb.init(project="opentable-vision", reinit=True)
# Log the table to visualize with a run...
run.log({"opentable_gptv": opentable_table_eligible})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

### Eval Metrics

In [ ]:
from autoevals.llm import *
from autoevals.string import Levenshtein
import warnings
Levenshtein()(['b'], ['a']).score

0.8

In [ ]:
config_metrics = {
    "Actions": {"edit": "edit"}
}
metric_registry = {
    "edit": Levenshtein()
}
def evaluate(dataset):
    """
    eligible dataset
    """
    metric_results = []
    for index, row in dataset[eligible].iterrows():
        result_row = {}
        for metric_config in config_metrics:
            metric_name = metric_config["name"]
            if metric_registry.get(metric_name):
                metric_func = metric_registry[metric_name]
                result_row[metric_name] = metric_func(row)
        metric_results.append(result_row)